<a href="https://colab.research.google.com/github/ykato27/Image-Captioning/blob/main/PyTorch%E3%82%92%E6%B4%BB%E7%94%A8%E3%81%97%E7%94%BB%E5%83%8F%E3%81%AE%E3%82%AD%E3%83%A3%E3%83%97%E3%82%B7%E3%83%A7%E3%83%B3%E3%82%92%E8%87%AA%E5%8B%95%E7%94%9F%E6%88%90%E3%81%99%E3%82%8B%E3%83%AC%E3%82%B7%E3%83%94ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install nltk
!pip3 install numpy
!pip3 install Pillow
!pip3 install pycocotools

In [2]:
import nltk
import pickle
import os
import numpy as np
from PIL import Image
from collections import Counter
from pycocotools.coco import COCO
import glob
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
import torch.utils.data as data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!mkdir data
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./data/
!wget http://images.cocodataset.org/zips/train2014.zip -P ./data/

!unzip ./data/captions_train-val2014.zip -d ./data/
!rm ./data/captions_train-val2014.zip
!unzip ./data/train2014.zip -d ./data/
!rm ./data/train2014.zip 

ストリーミング出力は最後の 5000 行に切り捨てられました。
 extracting: ./data/train2014/COCO_train2014_000000408557.jpg  
 extracting: ./data/train2014/COCO_train2014_000000013714.jpg  
 extracting: ./data/train2014/COCO_train2014_000000194043.jpg  
 extracting: ./data/train2014/COCO_train2014_000000219859.jpg  
 extracting: ./data/train2014/COCO_train2014_000000278135.jpg  
 extracting: ./data/train2014/COCO_train2014_000000141015.jpg  
 extracting: ./data/train2014/COCO_train2014_000000280923.jpg  
 extracting: ./data/train2014/COCO_train2014_000000200024.jpg  
 extracting: ./data/train2014/COCO_train2014_000000435713.jpg  
 extracting: ./data/train2014/COCO_train2014_000000249993.jpg  
 extracting: ./data/train2014/COCO_train2014_000000424160.jpg  
 extracting: ./data/train2014/COCO_train2014_000000142761.jpg  
 extracting: ./data/train2014/COCO_train2014_000000532668.jpg  
 extracting: ./data/train2014/COCO_train2014_000000564904.jpg  
 extracting: ./data/train2014/COCO_train2014_000000346384.jpg  
 extract

#### キャプションデータの前処理

In [4]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def make_vocab(json, threshold):
    coco = COCO(json)
    counter = Counter()
    ids = coco.anns.keys()
    for i, id in enumerate(ids):
        caption = str(coco.anns[id]['caption'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)

        if (i+1) % 1000 == 0:
            print("[{}/{}] Tokenized captions.".format(i+1, len(ids)))

    words = [word for word, cnt in counter.items() if cnt >= threshold]

    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

vocab = make_vocab(json='./data/annotations/captions_train2014.json', threshold=4)
vocab_path = './data/vocab.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved vocabulary wrapper to '{}'".format(vocab_path))

loading annotations into memory...
Done (t=2.65s)
creating index...
index created!
[1000/414113] Tokenized captions.
[2000/414113] Tokenized captions.
[3000/414113] Tokenized captions.
[4000/414113] Tokenized captions.
[5000/414113] Tokenized captions.
[6000/414113] Tokenized captions.
[7000/414113] Tokenized captions.
[8000/414113] Tokenized captions.
[9000/414113] Tokenized captions.
[10000/414113] Tokenized captions.
[11000/414113] Tokenized captions.
[12000/414113] Tokenized captions.
[13000/414113] Tokenized captions.
[14000/414113] Tokenized captions.
[15000/414113] Tokenized captions.
[16000/414113] Tokenized captions.
[17000/414113] Tokenized captions.
[18000/414113] Tokenized captions.
[19000/414113] Tokenized captions.
[20000/414113] Tokenized captions.
[21000/414113] Tokenized captions.
[22000/414113] Tokenized captions.
[23000/414113] Tokenized captions.
[24000/414113] Tokenized captions.
[25000/414113] Tokenized captions.
[26000/414113] Tokenized captions.
[27000/414113] T

#### 画像のリサイズ

In [5]:
def resize_image(image, size):
    return image.resize((size,size), Image.ANTIALIAS)

def resize_images(image_dir, output_dir, size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = os.listdir(image_dir)
    num_images = len(images)
    for i, image in enumerate(images):
        with open(os.path.join(image_dir, image), 'r+b') as f:
            with Image.open(f) as img:
                img = resize_image(img, size)
                img.save(os.path.join(output_dir, image), img.format)
        if (i+1) % 100 == 0:
            print ("[{}/{}] Resized the images and saved into '{}'."
                   .format(i+1, num_images, output_dir))

image_dir = './data/train2014/'
output_dir = './data/resized2014/'
image_size = 256
resize_images(image_dir, output_dir, image_size)

[100/82783] Resized the images and saved into './data/resized2014/'.
[200/82783] Resized the images and saved into './data/resized2014/'.
[300/82783] Resized the images and saved into './data/resized2014/'.
[400/82783] Resized the images and saved into './data/resized2014/'.
[500/82783] Resized the images and saved into './data/resized2014/'.
[600/82783] Resized the images and saved into './data/resized2014/'.
[700/82783] Resized the images and saved into './data/resized2014/'.
[800/82783] Resized the images and saved into './data/resized2014/'.
[900/82783] Resized the images and saved into './data/resized2014/'.
[1000/82783] Resized the images and saved into './data/resized2014/'.
[1100/82783] Resized the images and saved into './data/resized2014/'.
[1200/82783] Resized the images and saved into './data/resized2014/'.
[1300/82783] Resized the images and saved into './data/resized2014/'.
[1400/82783] Resized the images and saved into './data/resized2014/'.
[1500/82783] Resized the imag

#### データセットの作成

In [6]:
class CocoDataset(data.Dataset):
    def __init__(self, root, json, vocab, transform=None):
        self.root = root
        self.coco = COCO(json)
        self.ids = list(self.coco.anns.keys())
        self.vocab = vocab
        self.transform = transform

    def __getitem__(self, index):
        coco = self.coco
        vocab = self.vocab
        ann_id = self.ids[index]
        caption = coco.anns[ann_id]['caption']
        img_id = coco.anns[ann_id]['image_id']
        path = coco.loadImgs(img_id)[0]['file_name']

        image = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        return image, target

    def __len__(self):
        return len(self.ids)

#### データローダーの作成

In [7]:
def collate_fn(data):
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)

    images = torch.stack(images, 0)

    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]        
    return images, targets, lengths

def get_loader(root, json, vocab, transform, batch_size, shuffle, num_workers):
    coco = CocoDataset(root=root,
                       json=json,
                       vocab=vocab,
                       transform=transform)
    
    data_loader = torch.utils.data.DataLoader(dataset=coco, 
                                              batch_size=batch_size,
                                              shuffle=shuffle,
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)
    
    return data_loader

#### ニューラルネットワークモデルの作成

#### Encoderモデル

In [8]:
class Encoder(nn.Module):
    def __init__(self, embed_size):
        super(Encoder, self).__init__()
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
        
    def forward(self, images):
        with torch.no_grad():
            features = self.resnet(images)
        features = features.reshape(features.size(0), -1)
        features = self.bn(self.linear(features))
        return features

#### Decoderモデル

In [9]:
class Decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    def forward(self, features, captions, lengths):
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def predict(self, features, states=None):
        predicted_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)
            outputs = self.linear(hiddens.squeeze(1))
            _, predicted = outputs.max(1)
            predicted_ids.append(predicted)
            inputs = self.embed(predicted)
            inputs = inputs.unsqueeze(1)
        predicted_ids = torch.stack(predicted_ids, 1)
        return predicted_ids

#### pack_padded_sequence

1.pad_sequence

In [10]:
a = torch.full((3,3),fill_value=1)
b = torch.full((2,3),fill_value=2)
c = torch.full((1,3),fill_value=3)

print(a)
print(b)
print(c)

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])
tensor([[2, 2, 2],
        [2, 2, 2]])
tensor([[3, 3, 3]])


In [11]:
from torch.nn.utils.rnn import pad_sequence
d = pad_sequence([a,b,c], batch_first=True)
print(d)

tensor([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[2, 2, 2],
         [2, 2, 2],
         [0, 0, 0]],

        [[3, 3, 3],
         [0, 0, 0],
         [0, 0, 0]]])


2.pack_sequence

In [12]:
from torch.nn.utils.rnn import pack_sequence
e = pack_sequence([a,b,c])
print(e)

PackedSequence(data=tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [1, 1, 1],
        [2, 2, 2],
        [1, 1, 1]]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)


3.pack_padded_sequence

In [13]:
embeddings = pad_sequence([a,b,c], batch_first=True)
lengths = torch.tensor([3,2,1])
f = pack_padded_sequence(embeddings, lengths, batch_first=True)
print(f)

PackedSequence(data=tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [1, 1, 1],
        [2, 2, 2],
        [1, 1, 1]]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)


#### 学習

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = 'models/'
crop_size = 224
vocab_path = 'data/vocab.pkl'
image_dir ='data/resized2014'
caption_path='data/annotations/captions_train2014.json'
log_step=10
save_step=1000
embed_size=256
hidden_size=512
num_layers=1
num_epochs=10
batch_size=128
num_workers=2
learning_rate=0.001

if not os.path.exists(model_path):
    os.makedirs(model_path)

transform = transforms.Compose([ 
    transforms.RandomCrop(crop_size),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

data_loader = get_loader(image_dir, caption_path, vocab, transform, batch_size,
                          shuffle=True, num_workers=num_workers) 

encoder = Encoder(embed_size).to(device)
decoder = Decoder(embed_size, hidden_size, len(vocab), num_layers).to(device)

criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(params, lr=learning_rate)

total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, captions, lengths) in enumerate(data_loader):
        
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        
        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        loss = criterion(outputs, targets)
        decoder.zero_grad()
        encoder.zero_grad()
        loss.backward()
        optimizer.step()

        if i % log_step == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Perplexity: {:5.4f}'
                  .format(epoch, num_epochs, i, total_step, loss.item(), np.exp(loss.item()))) 
            
        if (i+1) % save_step == 0:
            torch.save(decoder.state_dict(), os.path.join(
                model_path, 'decoder-{}-{}.ckpt'.format(epoch+1, i+1)))
            torch.save(encoder.state_dict(), os.path.join(
                model_path, 'encoder-{}-{}.ckpt'.format(epoch+1, i+1)))

loading annotations into memory...
Done (t=2.44s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [0/10], Step [0/3236], Loss: 9.2047, Perplexity: 9943.7787
Epoch [0/10], Step [10/3236], Loss: 5.8110, Perplexity: 333.9379
Epoch [0/10], Step [20/3236], Loss: 5.3682, Perplexity: 214.4859
Epoch [0/10], Step [30/3236], Loss: 5.0210, Perplexity: 151.5623
Epoch [0/10], Step [40/3236], Loss: 4.6711, Perplexity: 106.8165
Epoch [0/10], Step [50/3236], Loss: 4.3994, Perplexity: 81.4026
Epoch [0/10], Step [60/3236], Loss: 4.2661, Perplexity: 71.2433
Epoch [0/10], Step [70/3236], Loss: 3.9944, Perplexity: 54.2932
Epoch [0/10], Step [80/3236], Loss: 3.9336, Perplexity: 51.0885
Epoch [0/10], Step [90/3236], Loss: 4.0545, Perplexity: 57.6556
Epoch [0/10], Step [100/3236], Loss: 3.8380, Perplexity: 46.4338
Epoch [0/10], Step [110/3236], Loss: 3.7475, Perplexity: 42.4134
Epoch [0/10], Step [120/3236], Loss: 3.6701, Perplexity: 39.2552
Epoch [0/10], Step [130/3236], Loss: 3.6097, Perplexity: 36.9536
Epoch [0/10], Step [140/3236], Loss: 3.6879, Perplexity: 39.9614
Epoch [0/10], Step [150/3236],

#### 推論

In [ ]:
!wget https://www.dropbox.com/s/ne0ixz5d58ccbbz/pretrained_model.zip
!unzip pretrained_model.zip
!mv encoder-5-3000.pkl decoder-5-3000.pkl models
!wget https://www.dropbox.com/s/26adb7y9m98uisa/vocap.zip
!unzip vocap.zip
!mv vocab.pkl models

#### 翻訳APIのインストール

In [ ]:
!pip uninstall googletrans
!pip install googletrans==4.0.0-rc1

#### 推論画像のダウンロード

In [ ]:
!wget 'https://tensorflow.org/images/surf.jpg'

#### 推論の実行|

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([224, 224], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

encoder_path ='models/encoder-5-3000.pkl'
decoder_path ='models/decoder-5-3000.pkl'
vocab_path ='models/vocab.pkl'

embed_size=256
hidden_size=512
num_layers=1

image_path = './surf.jpg'

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

encoder = Encoder(embed_size).eval()
decoder = Decoder(embed_size, hidden_size, len(vocab), num_layers)
encoder = encoder.to(device)
decoder = decoder.to(device)

encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

image = load_image(image_path, transform)
image_tensor = image.to(device)

feature = encoder(image_tensor)
predicted_ids = decoder.predict(feature)
predicted_ids = predicted_ids[0].cpu().numpy() 

predicted_caption = []
for word_id in predicted_ids:
    word = vocab.idx2word[word_id]
    predicted_caption.append(word)
    if word == '<end>':
        break
sentence = ' '.join(predicted_caption[1:-2])

from googletrans import Translator
translator = Translator()
translation = translator.translate(sentence, dest='ja')

image = Image.open(image_path)
plt.imshow(np.asarray(image))
plt.show()
print (sentence)
print(translation.text)